# Processes

Virtual memory is not real memory. Remember we have the stack vs heap.They grow together but in opposite directions. We take from the heap when we are dynamically creating memory, otherwise stack. Heaps are used for lists and pointers, and the stack is also used for recursion.



## Process concepts


Operating systems executes variety of programs
- Batch system (__jobs__)
- Time Shared Systems (__user programs or tasks__)

Job and process are interchangable terms

Processes are programs in execution, and must progress in a sequential fashion. __Programs are passive, Processes are active__.


Virtual memory is divided into multiple parts called execution contetxs. There is

- Text Section. Where the code resides
- Program Counter and CPU registers
- Stack. Contains temporary data like function params, return addresses, executes
- Data Section. Contains global variables
- Heap. Contains dynamically allocated memory

__ Virtual Memory is not the same as physical memory. __


A program is a __passive__ entity on the disk (usually as an executable file), and becomes an active process when we execute it.

### Process state

As the process executes, it changes its internal state.

- new: being created

- ready: waiting to be assigned to processor

- running: instr being executed

- waiting: waiting for event to occur

- terminated: process has finished executed


Interrupts come from software, signals come from hardware

<div style="text-align: center"><img class="center" src = Pictures/process_state_flowchart.png></img></div>


### Process Control Block 

This is created when a process is created and contains metadata about the process. Things such as 
- process state
- process ID
- program counter
- CPU registers
- memory info
- accounting info (system stats)
- I/O status information


Process n will run untill a system call. Save state into PCBn, then we idle process n for however long. the reload statve from PCBn+1 then do process n+1. Then reload state from PCBn and continue process n

<div style="text-align: center"><img class="center" src = Pictures/cpu_pcb_flowchart.png></img></div>





> Generally, what is the pcb for, when is it accessed?

The PCBs purpose is to allow for a sort of book mark for the progress of a process when we need to switch processes. This lets us multitask on a thread.

### Fork and Exec


Fork is used to create a child process, whereas exec is used to switch the process. Fork will create a child process that is a duplicate of the parent in virtual memory with its own heap and stack, and then we can use exec to swicth to the process that we want to run. If we didn't use fork, we could only have one process and couldn't be able to multitask.



### Fork

Creates a new process and __returns two return codes__. When the rc > 0, we know we are running in the parent process, and the returned rc is the PID of the child. However when the returned rc = 0, we know were in the child process. If rc = 0, then we have some error and we stop the process.



## Process creation

Parent process create child processes and so on. Creatning a tree of processes. Identified by a PID

Resource sharing options

parent and child share same resources or children share subset of parents resources, or resources are not shared between parent and child.


Address space
child is duplicate of parent, then program is loaded into child

fork() exec()

systemd or init are the first process started, with PID = 1


## Process Termination
Process executes last statement and then asks operating system to delete it by using the exit() system call. Child returns data to os via wait() sys call, processes resources are deallocated by OS. If children process are causing errors, we can use the abort syscall.
- exceeded allocated resources, task is no longer required, parent exits, but child was still running

- cascadeing termination, kindof like recursively freeing a list

`pid = wait(&status)` returns status info nd pid of terminated process. Use by parent to wait for termination of child process.

zombie process - when no parent is waiting, then process is zombie. Processes that are no longer doing productive work. When a child terminates, it will return an exit code, that the parent must read before the child is considered fully terminated. The time in between when the process stops making productive work, and the time that it is reaped, it is considered a zombie. oNly occupy space in the process table. They take no space in memory or cpu.

orphan - when the parent was terminated without invoking wait. Orphanned children processes are immediately adopted by systemd. Init automatically reaps its children when appropriate. Parents parent will adopt orhan before init will. If no available kin, then init will take it. Orphans can be ready, running, or in waiting.

either way parent terminated before child.

### Process Scheduling

The process scheduler selects among next available processes for scheduling queues. This is done to maximize CPU execution for time sharing. Thee available scheduling queues are the

- Job Queue: List of all processes in the system
- Ready Queue: Set of all processes residing in main memory, ready and waiting to be executed.
- Device Queue: Set of processes waitign for an I/O device

Processes can switch between different scheduling queues, as seen below.

<div style="text-align: center"><img class="center" src = Pictures/switching_sched_queues.png></img></div>


Below is a queueing diagram for scheduling, represents other queues, proceses, resources or flows. Rectangular boxes are queues.

<div style="text-align: center"><img class="center" src = Pictures/scheduling_diagram.png></img></div>


The short term scheduler selects which process to be executed next, and the long terms scheduler
selects which process to be brought into ready queue. Controlls degreee of multiprogramming. The long term scheduler wants a good diversity of processes to be exectured o they aren't bottlenecking resources and can be done quicker.


Process categories
- io processes
- cpu bound process
- mobile devices


### Mobile Processes

Multitasking in mobile systems allow only one process to run, while the others are suspended. IOS provides for a single foreground process which is controlled via user interface, while we also have multiple background processes in memory but not on the display.

## Multiprocess architecture

Example as a browser

Chrome has 3 main processes

Browser - Handles UI, disk, network, and I/O
Renderer - processes and renders web pages. HTML and javascript. This runs in a sandbox, which containerizes IO and hardware resources.
Plug in - process for each type of plug in

## The Producer Consumer Problem

This is a paradigm for cooperating processes. The producer process produces information consumed by the consumer process. Producers put data a buffer and consumers take it out. Both processes must cooperate, communicate, and synchronize. An __unbounded buffer__ has no practical limit on size of the buffer, while bounded buffers assume that there is a fixed size limit. The producer needs to know if there is room in the buffer, and if not, it won't try to put more data there.


Lucy is a bounded buffer because she can only wrap so much at a time. Coordination between her and the manager is needed.
<div style="text-align: center"><img src="Pictures/lucy.png"></div>


## Interprocess communication

Processes may be independent or cooperating. Cooperating resources can be affected by other processes including sharing data. Cooperating resources need __Interprocess communication (IPC)__. We need to know why we would want to differentiate between these IPC methods

- Message Passing

    <div style="text-align: center"><img src="Pictures/message_passing.png"></div>

    This is the memory used for two processes A and B. A will stick data in the message queue and B will call it when appropriate. The memory for this message queue lies in the kernel. Usues a system call to access this memory. Since this is a queue, does this mean that communication is one way. One may prefer message passing if the data is not hyper time sensitive or because coordination between proceses is much easier with message passing.

    ### Synchronization

    Message passing may be either blocking or non blocking. 

    Blocking is considered synchronous
    - Blocking Send -- the sender is blocked until the message is recieved.
    - Blocking Recieve -- the reciever is blocked until a message is available.

    Non-blocking is considered asynchronous
    - Non-blocking Send -- the sender sends the message and continues
    - Non-blocking Recieve -- the reciever recieves either a valid or null message
    
    We can have a combination of types for the sender and reciever, and if they are both blocking, then it is called a rendezvous

- Shared Memory

    <div style="text-align: center"><span><img src="Pictures/shared_memory.png"></span></div>

    Here the memory which holds the message is __not__ in the kernel. They each have access to an allocation of shared memory user space? How does B know that there is data from A in the memory. One may prefer message passing to shared memory because it takes time to tranfer data back and forth between user and kernal land. The communication is under control of the user processes, not the OS. No system calls are used, other than setting up the processes, which is why its fast. 

    `ftok` - convert a pathname and project identifier to a System V IPC key

- Sockets

    Sockets are an endpoint for communication. They are a concatenation of an IP address and a port number. It is a number included at the start of a message packet to differentiate network services on a host. Communication happens between a pair of sockets.

    Localhost (127.0.0.1) is a loopback socket which points to the machine of the currently executing process.

- Remote Procedure Call

    These abstract procedure calls between processes on networked systems (uses ports for differentiation). 

    - Stubs -- client side proxy for the actual procedure on the server. CLient side stub locates the server and __marshalls__ the parameters. The server side stub recieves the message, unpacks paramters sent by client, and performs procedures sent by client.
    
- Pipes

    Acts as conduit between processes to comunicate on the same computer. Pipes are unidirectional.

    - Ordinary Pipes -- cannot be accessed through outside the process that created it. Parent process creates an ordinary pipe and uses it to communicate with a child process that it created in a standard producer consumer relationship. Producer writes to write end of pipe and consumer reads from read end of the pipe. Called anonymous pipes in Windows. Done with `proc_a | proc_b`

    - Named Pipes -- can be accessed without a parent-child relationship. Created with `mkfifo <pipe_name>`

        These can act like blocking senders and recievers, where the sender will wait for the reciever to clear the pipe, or when a reciever currently looking at an empty pipe will wait for the next message to come down the pipe. Can be bidirectional. Names pipes do not persist as they are stored in a virtual file. This is why they technically have 0 bytes.


    __How are pipes different from something like shared memory?__

__Independent processes__ cannot be affected by the execution of another process. They do not share a state. __Cooperating processes__ can affect or be affected by the execution of another process. Shares state and is subject to race conditions.